In [15]:
from datasets import load_dataset

dataset = load_dataset("cfahlgren1/react-code-instructions", split="train")

def process_example(example):
    message_user = ""
    message_assistant = ""
    
    for message in example["messages"]:
        if message["role"] == "user":
            message_user += message["content"] + "\n"
        elif message["role"] == "assistant":
            message_assistant += message["content"] + "\n"
    
    return {"prompt": message_user, "output": message_assistant}

dataset_cleaned = dataset.map(process_example)

print(f"Número de ejemplos: {len(dataset_cleaned)}")
print(dataset_cleaned[0])

Map: 100%|██████████| 74428/74428 [00:04<00:00, 18521.00 examples/s]

Número de ejemplos: 74428
{'created_at': '2025-01-06 05:09:57.778612', 'model': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'messages': [{'role': 'system', 'content': "\nYou are an expert React, TypeScript, and TailwindCSS developer with a keen eye for modern, aesthetically pleasing design.\n\nYour task is to create a stunning, contemporary, and highly functional website based on the user's request using a SINGLE static React JSX file, which exports a default component. This code will go directly into the App.tsx file and will be used to render the website.\n\nGeneral guidelines:\n- Ensure the React app is a single page application with a cohesive design language throughout.\n- DO NOT include any external libraries, frameworks, or dependencies outside of what is already installed.\n- For icons, create simple, elegant SVG icons. DO NOT use any icon libraries.\n- Utilize TailwindCSS for styling, focusing on creating a visually appealing and responsive layout.\n- Avoid using arbitrary values (e.g.

In [24]:
split_dataset = dataset_cleaned.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print(train_dataset)
print(val_dataset)

Dataset({
    features: ['created_at', 'model', 'messages', 'recommended', 'upvoted', 'prompt', 'output'],
    num_rows: 59542
})
Dataset({
    features: ['created_at', 'model', 'messages', 'recommended', 'upvoted', 'prompt', 'output'],
    num_rows: 14886
})


In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/codegemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`